In [ ]:
import fastai

from fastai import *
from fastai.vision import *
from fastai.text import *

from torchvision.models import *
import pretrainedmodels

import numpy as np 
import pandas as pd

from fastai.utils import *
from torch.utils import *

from fastai.callbacks import *
from sklearn.model_selection import train_test_split

In [ ]:
DATA_ROOT = '../data/toxic_comment/cleaned/'

In [ ]:
train_df = pd.read_csv(DATA_ROOT + 'train.csv')
test_df = pd.read_csv(DATA_ROOT + 'test.csv')

In [ ]:
# split 8:2 ratio
train_df.fillna('no comment', inplace=True)
test_df.fillna('no comment', inplace=True)
train, val = train_test_split(
    train_df, shuffle=True, test_size=0.2, random_state=42
)

In [ ]:
src_lm = ItemLists(
    '.', TextList.from_df(train, path=".", cols = "comment_text"),
    TextList.from_df(val, path=".", cols = 'comment_text')
)

In [ ]:
data_lm = src_lm.label_for_lm().databunch(bs=32)

In [ ]:
data_lm.show_batch()

In [ ]:
learn = language_model_learner(
    data_lm, AWD_LSTM, drop_mult=0.3,
    model_dir="fast_ai_model/"
)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(
    1, max_lr=slice(5e-4, 5e-3), moms=(0.8, 0.7), pct_start=0.2,
    wd =(1e-7, 1e-5, 1e-4, 1e-3)
)

In [ ]:
learn.save('fit_head')
learn.load('fit_head')

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(
    10, max_lr=slice(1e-4, 1e-3), moms=(0.8, 0.7), pct_start=0.2,
    wd=(1e-7, 1e-5, 1e-4,  1e-2)
)

In [ ]:
learn.save('fine-tuned')
learn.load('fine-tuned')
learn.save_encoder('fine-tuned')

In [ ]:
TEXT = "He is a piece of"
N_WORDS = 10
N_SENTENCES = 2

In [ ]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
learn.export()